### Проект 4 - Подготовка рекомендательной системы для сайта 

КЕЙС 
Онлайн-школа MasterMind стремится к увеличению среднего чека за счет рекомендательной системы на сайте. При оформлении корзины пользователь должен получить рекомендацию еще на два курса. 

ЦЕЛЬ
Подготовить основу рекомендательной системы.

ЗАДАЧИ
1. Проанализировать базу данных о продажах за 2017-2018 гг. с помощью SQL
2. Выгрузить из БД необходимые данные
3. Обработать данные в Python
4. Получить рекомендательную таблицу






In [449]:
import pandas as pd #импортируем все необходимые библиотеки и модули
import itertools
import numpy as np
import psycopg2
from collections import Counter
from collections import defaultdict


 ### Комментарий к коду SQL


    with course_count as(             - CTE где отображены все пользователи купившие курсы и кол-во купленных ими курсов
    select 
    distinct c.user_id,
    count(distinct(ci.resource_id)) total_courses
    from final.carts as c
    join final.cart_items as ci on c.id=ci.cart_id
    where c.state='successful' and ci.resource_type='Course'
    group by c.user_id)                                      


    select                                      - Основной запрос: все уникальные пользователи и номера курсов              
    distinct(cc.user_id),
    ci.resource_id
    from final.carts as c 
    join course_count as cc on cc.user_id=c.user_id
    join final.cart_items as ci on ci.cart_id=c.id
    where cc.total_courses>=2 and                 - Только пол-ти купившие больше 2-ух курсов
    c.state='successful' and ci.resource_type='Course'
    order by 1                                       

In [450]:
# запрос к БД содержащий SQL код
try: 
    connection = psycopg2.connect( #параметры подключения к БД
        dbname='skillfactory', 
        user='skillfactory',
        host='84.201.134.129',
        password='cCkxxLVrDE8EbvjueeMedPKt',
        port=5432,
    )
    cursor = connection.cursor()
    #SQL код
    postgres_select_query = """ with course_count as(
    select 
    distinct c.user_id,
    count(distinct(ci.resource_id)) total_courses
    from final.carts as c
    join final.cart_items as ci on c.id=ci.cart_id
    where c.state='successful' and ci.resource_type='Course'
    group by c.user_id)
    select 
    distinct(cc.user_id),
    ci.resource_id
    from final.carts as c 
    join course_count as cc on cc.user_id=c.user_id
    join final.cart_items as ci on ci.cart_id=c.id
    where cc.total_courses>=2 and ci.resource_type='Course' and c.state='successful' 
    order by 1 """
    cursor.execute(postgres_select_query)
    courses_df=cursor.fetchall()
    connection.commit()
   
except (Exception, psycopg2.Error) as error:
    if connection:
        print("Error", error)
finally:
    if connection:
        cursor.close()
        connection.close()

In [451]:
courses_df = pd.DataFrame(courses_df) # преобразуем в DataFrame
courses_df=courses_df.rename(columns={0:'user_id',1:'resource_id'}) #переименовываем столбцы 


In [452]:
courses_df.info() #проанализируем данные в полученной DF, тип данных, кол-во пропусков

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34074 entries, 0 to 34073
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      34074 non-null  int64
 1   resource_id  34074 non-null  int64
dtypes: int64(2)
memory usage: 532.5 KB


In [453]:
print(courses_df['user_id'].nunique()) #проверим кол-во уникальных пользователей
print(courses_df['resource_id'].nunique()) #проверим кол-во уникальных курсов

12656
126


In [454]:
courses_df.head() # шапка DF

,user_id,resource_id
0,51,516
1,51,1099
2,6117,356
3,6117,357
4,6117,1125


In [455]:
courses_pivot = courses_df.groupby(['user_id'])['resource_id'].apply(list) #сгруппируем данные по id пользователя
courses_pivot = courses_pivot.to_frame(name='list_of_courses')             # новая DF, где каждому пользователю соответствует
courses_pivot.head()                                                       # список купленных им курсов
            

,list_of_courses
user_id,
51,"[516, 1099]"
6117,"[356, 357, 1125]"
10275,"[553, 1147]"
10457,"[361, 1138]"
17166,"[356, 357]"


In [456]:
def get_pairs(data): # функция для генерации всех комбинаций пар курсов
    list_of_pairs=[]
    for i in data['list_of_courses'].values:
        list_of_pairs.append(list(itertools.combinations(i, 2)))
    return list_of_pairs
pairs = get_pairs(courses_pivot) # используем функцию


In [457]:
pairs_new = [] #преобразуем двухуровневый список в одноуровневый  
for pair in pairs:
        pairs_new.extend(pair)


In [458]:
print(len(Counter(pairs_new).keys())) #посчитаем кол-во уникальных пар (курсов купленных вместе)
print(Counter(pairs_new).most_common(1)) #отобразим самую популярную пару курсов


3989
[((551, 566), 797)]


In [459]:
recommend_dict = defaultdict(list) # создаем Defaultdict для накопления и подсчета всех сочетаний в парах для каждого курса 
for course,rec in pairs_new:
    recommend_dict[course].append(rec)
recommend_dict = dict(recommend_dict) #преобразуем в словарь

In [460]:
len(recommend_dict.keys()) #проверяем для какого кол-ва курсов есть данные

117

In [461]:
# так как изначально кол-во курсов было 126, найдем "потерянные" при подсчете курсы, т.е. курсы, для которых не было рекомендаций

In [462]:
list_of_missing_courses = set(courses_df['resource_id'].unique() )-set(recommend_dict.keys())
list_of_missing_courses  # зафиксируем "потерянные" курсы, чтобы в дальнейшенм вставить их в таблицу

{1184, 833, 1188, 902, 1160, 1198, 1199, 1200, 1201}

In [463]:
rec_dict={} #создаем словарь, заполняем данными: ключ - номер курса, значение - список кортежей (номер парного курса, популярность)
for key,value in recommend_dict.items():
    rec_dict[key]=Counter(value).most_common(2) #в словарь попадают только 2 самые популярные пары курсов

In [464]:
for item in list_of_missing_courses: #добавляем "потерянные" курсы в качестве ключа с пустыми данными в качестве значений
    rec_dict[item]=[(0,0),(0,0)]
    

In [465]:
for key,value in rec_dict.items(): #заполняем "пустыми" кортежами (0,0) пропуски в данных 
    if len(value)<2:               #курсы, для которых существует менее 2-ух рекомендаций
        value.append((0,0))

In [466]:
recommend_df = pd.DataFrame.from_dict(rec_dict,orient='index',columns=['recomm_1','recomm_2']).reset_index() #преобразуем словарь в DataFrame

In [467]:
recommend_df['rec_1']= recommend_df.recomm_1.apply(lambda x:x[0]) #разделяем данные из кортежей в отдельные столбцы для каждого курса
recommend_df['popul_1']= recommend_df.recomm_1.apply(lambda x:x[1])
recommend_df['rec_2']= recommend_df.recomm_2.apply(lambda x:x[0])
recommend_df['popul_2']= recommend_df.recomm_2.apply(lambda x:x[1])
recommend_df=recommend_df.drop(['recomm_1','recomm_2'],axis=1)    #удаляем лишние столбцы
recommend_df=recommend_df.rename(columns={'index':'resource_id'}) #переименовываем столбец


In [468]:
recommend_df # таблица рекомендаций требует дополнений. Многие курсы не имеют рекомендаций (одной или двух), для некоторых
# курсов кол-во рекомендаций слишком мало


,resource_id,rec_1,popul_1,rec_2,popul_2
0,516,745,199,553,107
1,356,571,103,357,100
2,357,571,112,1125,52
3,553,745,212,568,83
4,361,551,49,1138,40
...,...,...,...,...,...
121,1160,0,0,0,0
122,1198,0,0,0,0
123,1199,0,0,0,0
124,1200,0,0,0,0


In [469]:
frequency_bottom_line_1=np.quantile(recommend_df['popul_1'],0.25) #определим значение квартиля Q1 для столбцов "популярности"
frequency_bottom_line_2=np.quantile(recommend_df['popul_2'],0.25) # примем эти значения как нижние границы частотности

In [470]:
most_popular_1 = recommend_df.groupby(['rec_1'])['popul_1'].sum().sort_values(ascending=False) #определим самые популярные курсы
most_popular_2 = recommend_df.groupby(['rec_2'])['popul_2'].sum().sort_values(ascending=False) # в каждом столбце "популярности"
most_popular_1.head(5)                                                                         


rec_1
551    1810
566    1133
745     614
572     429
794     414
Name: popul_1, dtype: int64

In [471]:
most_popular_2.head(5)

rec_2
515    508
552    321
523    313
551    267
752    212
Name: popul_2, dtype: int64

In [472]:
# Самыми покупаемыми курсами являются: 551 и 566 (популярность превышает ближайшие данные в 3 и 2 раза соответственно).
# Данные курсы мы будем рекомендовать для всех данных,где частотность слишком мала и в пропусках 

In [473]:
def insert_recom_1 (row):   #функции для замены значений ниже границы частотности и пропусков
    if row['popul_1']<=frequency_bottom_line_1:
        return 551
    else:
        return row['rec_1']
    
def insert_recom_2 (row):
    if row['popul_2']<=frequency_bottom_line_2:
        return 566
    else:
        return row['rec_2']
    

In [474]:
recommend_df['recommendation_1'] = recommend_df.apply(insert_recom_1,axis=1) # применим функции для замены значений
recommend_df['recommendation_2'] = recommend_df.apply(insert_recom_2,axis=1)
recommend_df = recommend_df.drop(['rec_1','rec_2','popul_1','popul_2'],axis=1) # удалим лишние столбцы

In [475]:
recommend_df # финальная таблица с рекомендациями

,resource_id,recommendation_1,recommendation_2
0,516,745,553
1,356,571,357
2,357,571,1125
3,553,745,568
4,361,551,1138
...,...,...,...
121,1160,551,566
122,1198,551,566
123,1199,551,566
124,1200,551,566


### Комментарий
В результате работы составлена таблица, где для каждого курса подобраны две рекомендации. Рекомендации найдены на основе анализа данных о продажах за 2017-2018 гг, из которых были выделены самые популярные сочетания курсов (курсы купленные одним пользователем).


В таблицу попали курсы с недостаточными данными (менее двух рекомендаций) или с очень низкими показателями популярности. Для этих случаев была определена нижняя граница частотности - первый квартиль столбцов популярности. Также найдены самые покупаемые в парах курсы - 551 и 566. Данные курсы добалены в качестве рекомендаций для всех курсов находящихся ниже границы частотности.

### Вывод

Полученная таблица готова для внедрения на сайт онлайн-школы. Для определения эффективности необходимо запустить А/В тесты.